# Recherche d'information : librairie PyTerrier

Dans cette partie, nous nous intéressons à la librairie [PyTerrier](https://pyterrier.readthedocs.io/en/latest/#) qui permet de mettre en place diverses briques d'un moteur de recherche.
PyTerrier est basée sur [Terrier](http://terrier.org/) qui est un moteur de recherche développé en Java.

Nous allons voir : 
*   l'installation et la configuration
*   l'indexation d'une collection
*   l'accès à l'index
*   l'évaluation d'un moteur de recherche


## Installation ete configuration

Après l'installation de la librairie, il est nécessaire d'initialiser PyTerrier pour importer les fichiers jar et démarrer la machine virtuelle associée; 

In [1]:
!pip install python-terrier

import pyterrier as pt
if not pt.started():
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.4/104.4 kB 2.4 MB/s eta 0:00:000:00:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 4.4 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 kB 5.8 MB/s eta 0:00:00
  Using cached Jinja2-3.1.2-py3-none-any.whl (133 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 5.7 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 8.0 MB/s eta 0:00:00m eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 8.0 MB/s eta 0:00:00
  P

PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


## Indexation d'une collection

Il est possible d'indexer plusieurs formats de collection : format TREC, fichiers en texte brut ou en PDF, ou encore des Dataframe Pandas ([pour plus de détails](https://pyterrier.readthedocs.io/en/latest/terrier-indexing.html)).

Un petit exemple à titre illustratif est foourni dans le code suivant :

In [2]:
import pandas as pd

# configuration de l'affichage
pd.set_option('display.max_colwidth', 150)

# le jeu de données au format DataFrame
docs_df = pd.DataFrame([
        ["d1", "this is the first document of many documents"],
        ["d2", "this is another document"],
        ["d3", "the topic of this document is unknown"]
    ], columns=["docno", "text"])

# création de l'index
indexer = pt.DFIndexer("./index_3docs", overwrite=True)         # Définition du format de données (DFIndexer())
index_ref = indexer.index(docs_df["text"], docs_df["docno"])    # Indexation des champs text et docno
!ls -lh index_3docs/                                            # Affichage de l'index sauvegardé dans "./index_3docs/"

/home/charles/.pyenv/versions/3.10.10/lib/python3.10/site-packages/pyterrier/index.py:628: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, value in meta_column[1].iteritems():


total 40K
-rw-r--r--. 1 charles charles    3 30 mars  14:01 data.direct.bf
-rw-r--r--. 1 charles charles   51 30 mars  14:01 data.document.fsarrayfile
-rw-r--r--. 1 charles charles    4 30 mars  14:01 data.inverted.bf
-rw-r--r--. 1 charles charles  344 30 mars  14:01 data.lexicon.fsomapfile
-rw-r--r--. 1 charles charles  249 30 mars  14:01 data.lexicon.fsomaphash
-rw-r--r--. 1 charles charles   33 30 mars  14:01 data.meta-0.fsomapfile
-rw-r--r--. 1 charles charles   24 30 mars  14:01 data.meta.idx
-rw-r--r--. 1 charles charles   48 30 mars  14:01 data.meta.zdata
-rw-r--r--. 1 charles charles 4,2K 30 mars  14:01 data.properties


De nombreux fichiers sont créés : index direct, index inverse, méta-données de l'index et de la configuration de l'indexation, etc...


Il est également possible de modifier la configuration de l'indexation : [voir ici](https://pyterrier.readthedocs.io/en/latest/terrier-indexing.html#indexing-configuration).

Pour chager un index existant en local :

In [3]:
index = pt.IndexFactory.of(index_ref)

Il est aussi possible de voir les statistiques de l'index. 
pour connaître toutes les fonctions d'interrogation, se référencer à la [javadoc](http://terrier.org/docs/current/javadoc/org/terrier/structures/Index.html).

In [4]:
# statistiques de la collection
print(index.getCollectionStatistics().toString())

Number of documents: 3
Number of terms: 4
Number of postings: 6
Number of fields: 0
Number of tokens: 7
Field names: []
Positions:   false



In [5]:
# statistiques du vocabulaire.
# Nt : document frequency : nombre de documents contenant le terme
# TF : term frequency : nombre d'occurences du terme
# maxTF : nombre d'occurences maximal pour un document
for kv in index.getLexicon():
  print("%s (%s) -> %s (%s)" % (kv.getKey(), type(kv.getKey()), kv.getValue().toString(), type(kv.getValue()) ) )

document (<class 'str'>) -> term0 Nt=3 TF=4 maxTF=2 @{0 0 0} (<class 'jnius.reflect.org.terrier.structures.LexiconEntry'>)
first (<class 'str'>) -> term1 Nt=1 TF=1 maxTF=1 @{0 0 7} (<class 'jnius.reflect.org.terrier.structures.LexiconEntry'>)
topic (<class 'str'>) -> term2 Nt=1 TF=1 maxTF=1 @{0 1 1} (<class 'jnius.reflect.org.terrier.structures.LexiconEntry'>)
unknown (<class 'str'>) -> term3 Nt=1 TF=1 maxTF=1 @{0 1 5} (<class 'jnius.reflect.org.terrier.structures.LexiconEntry'>)


In [6]:
# focus sur un terme particulier
index.getLexicon()["document"].toString()

'term0 Nt=3 TF=4 maxTF=2 @{0 0 0}'

In [7]:
# récupère les statistiques de l'index inverse à partir d'un terme particulier
pointer = index.getLexicon()["document"]
for posting in index.getInvertedIndex().getPostings(pointer):
    print(posting.toString() + " doclen=%d" % posting.getDocumentLength())

ID(0) TF(2) doclen=3
ID(1) TF(1) doclen=1
ID(2) TF(1) doclen=3



De plus, PyTerrier met à disposition [une collection de jeux de données pré-traités](https://pyterrier.readthedocs.io/en/latest/datasets.html).
Dans ce qui suit, nous allons nous concentrer sur le jeu de données CORD19 qui recense des articles liés à la crise sanitaire Covid-19. Il est

In [22]:
import os

cord19 = pt.datasets.get_dataset('irds:cord19/trec-covid')
pt_index_path = './terrier_cord19'

if not os.path.exists(pt_index_path + "/data.properties"):
  # création de l'index. Utilisation de l'itérateur pour parcourir la collection
  indexer = pt.index.IterDictIndexer(pt_index_path)

  # on donne à l'index la fonction pour parcourir l'index avec l'itérateur  get_corpus_iter() 
  # On spécifie les champs à indexer et les meta-données à sauvegarder
  index_ref = indexer.index(cord19.get_corpus_iter(), 
                            fields=('abstract',), 
                            meta=('docno',))

else:
  # dans le cas où l'index existe déjà
  index_ref = pt.IndexRef.of(pt_index_path + "/data.properties")
index = pt.IndexFactory.of(index_ref)

**Exercice 1**

Afficher les statistiques de l'index Cord19 et analyser statistiques du terme "tv" (pas trop fréquent pour question d'affichage).

In [35]:
lex = index.getLexicon()
le = lex.getLexiconEntry('tv')
le.toString()

'term18759 Nt=70 TF=180 maxTF=2147483647 @{0 18614194 0} TFf=180'

## Recherche de documents à partir de l'index

Pour effectuer une recherche dans l'index, il faut utiliser la fonction batchRetrieve qui prend en paramètre l'index et le modèle de pondération (tf, tf-idf, etc...). La liste des modèles supportés est disponible [ici](http://terrier.org/docs/current/javadoc/org/terrier/matching/models/package-summary.html).

In [9]:
br = pt.BatchRetrieve(index, wmodel="Tf")
br.search("chemical reactions")

,qid,docid,docno,rank,score,query
0,1,12409,ij3ncdb6,0,21.0,chemical reactions
1,1,24432,094bk0t0,1,13.0,chemical reactions
2,1,11268,qroxmo85,2,12.0,chemical reactions
3,1,120041,pb00yr0r,3,12.0,chemical reactions
4,1,147193,ei4rb8fr,4,12.0,chemical reactions
...,...,...,...,...,...,...
995,1,96252,8clnrlec,995,2.0,chemical reactions
996,1,96758,rvw94nsg,996,2.0,chemical reactions
997,1,97138,lrx94c81,997,2.0,chemical reactions
998,1,98181,w3160mgp,998,2.0,chemical reactions


On récupère alors un DataFrame dont les colonnes sont les suivantes : 
*   qid : identifiant de la requête. Par défaut, il s'agit de "1", puisqu'il s'agit de notre première et unique requête.
*   docid : l'identifiant interne de Terrier pour chaque document
*   docno : l'identifiant unique externe (chaîne de caractères) pour chaque document
*   score : score des documents selon le modèle choisi (ici : fréquence totale des tf des termes de la requête dans chaque document)
*   rank : rang du document dénotant l'ordre décroissant par score.
*   query : la requête d'entrée

In [10]:
# autre exemple de modèle : TF-IDF
tfidf = pt.BatchRetrieve(index, wmodel="TF_IDF")
tfidf.search("chemical reactions")

,qid,docid,docno,rank,score,query
0,1,18717,iavwkdpr,0,11.035982,chemical reactions
1,1,171636,v3blnh02,1,10.329726,chemical reactions
2,1,147193,ei4rb8fr,2,10.317138,chemical reactions
3,1,121217,msdycum2,3,9.653734,chemical reactions
4,1,170863,sj8i9ss2,4,9.500211,chemical reactions
...,...,...,...,...,...,...
995,1,2428,38aabxh1,995,3.790183,chemical reactions
996,1,14752,u709r8ss,996,3.790183,chemical reactions
997,1,20074,wxi1xsbo,997,3.790183,chemical reactions
998,1,117156,ts3obwts,998,3.790183,chemical reactions


On peut aussi fournir plusieurs requêtes grâce à un dataFrame. Pour interroger l'index, on applique la fonction transform() au BatchRetriever (br).
pour plus de détails, voir [les propriétés des transformations](https://pyterrier.readthedocs.io/en/latest/transformer.html) ainsi que les [opérations possibles](https://pyterrier.readthedocs.io/en/latest/operators.html).

In [38]:
import pandas as pd
queries = pd.DataFrame([["q1", "document"], ["q2", "first document"]], columns=["qid", "query"])
br.transform(queries)       # ou aussi : br(queries)

TypeError: BatchRetrieve.transform() takes 2 positional arguments but 3 were given

**Exercice 2**

Ordonnancer les documents pour 3 requêtes : "covid disease", "hospital" et "home".
La fonction d'ordonnacement devra être de la forme suivante : 


```
0.4 * score_Bm25 + 0.6 * score_Dirichlet
```


In [76]:
queries = pd.DataFrame([["q1", "covid disease"], ["q2", "hospital"], ["q3", "home"]], columns=["qid", "query"])
br_bm25 = pt.BatchRetrieve(index, wmodel="BM25")
br_dirichlet = pt.BatchRetrieve(index, wmodel="DirichletLM")
res1 = br_bm25(queries)
res2 = br_dirichlet(queries)
res = res1.merge(res2, on=["qid", "docno"])
res["score"] = 0.4 * res["score_x"] + 0.6 * res["score_y"]
res.sort_values(by="score", ascending=False).groupby(by="query_x").head(10)

,qid,docid_x,docno,rank_x,score_x,query_x,docid_y,rank_y,score_y,query_y,score
1053,q3,73694,tunpb48l,2,11.241837,home,73694,0,4.599916,home,7.256685
1051,q3,37219,el231qvh,0,11.454789,home,37219,1,4.323170,home,7.175818
1052,q3,71618,0tj6zv5q,1,11.349913,home,71618,2,4.314538,home,7.128688
1054,q3,36315,3gsczkua,3,11.233091,home,36315,3,4.024548,home,6.907965
1055,q3,160321,9yfjmayh,4,11.118108,home,160321,4,3.907676,home,6.791849
1063,q3,92571,oko0od3i,12,10.934236,home,92571,5,3.895758,home,6.711149
1058,q3,96063,3t4hs9yd,7,11.115848,home,96063,13,3.656233,home,6.640079
1059,q3,175096,d2f7kfmv,8,11.115848,home,175096,14,3.656233,home,6.640079
1057,q3,96062,1ub7mcvx,6,11.115848,home,96062,12,3.656233,home,6.640079
1060,q3,28125,wyuz1az9,9,11.073574,home,28125,24,3.507076,home,6.533675


In [64]:
res = ((0.4 * br_bm25 + 0.6 * br_dirichlet)%1000).transform(queries)
res.groupby(by="query").head()

,qid,docid,docno,score,query,rank
0,q1,43498.0,9d8go2sl,3.067285,covid disease,1
1,q1,106017.0,a0w3n30z,3.067285,covid disease,2
2,q1,106018.0,smyp9fyo,3.067285,covid disease,3
3,q1,106019.0,vtfn9y5p,3.067285,covid disease,4
4,q1,158611.0,30ff3xg2,3.370553,covid disease,0
1013,q2,50870.0,z36273gl,3.510357,hospital,12
1014,q2,6331.0,b04lq3b2,3.617898,hospital,2
1015,q2,157868.0,alcq4jf8,3.897342,hospital,0
1016,q2,172041.0,hbhtbgzd,3.613481,hospital,4
1017,q2,30059.0,ccod8byi,3.641884,hospital,1


**Excercice 3**

Créez un ordonnanceur qui effectue les opérations suivantes :
* obtient les 10 documents les mieux notés par fréquence de terme (wmodel="Tf")
* obtenir les 10 documents les mieux notés par TF.IDF (wmodel="TF_IDF")
* ré-ordonne uniquement les documents trouvés dans les DEUX paramètres de recherche précédents en utilisant BM25.

Combien de documents sont récupérés par ce pipeline complet pour la requête "chemical"?

Vérification : le document avec le docno "37771" devrait avoir un score de 12.426309 $ pour la requête "chemical".

In [78]:
def my_ordonancer(index, query_df):
    br_tf = pt.BatchRetrieve(index, wmodel="Tf") % 10
    br_tf_idf = pt.BatchRetrieve(index, wmodel="TF_IDF") % 10
    br_BM25 = pt.BatchRetrieve(index, wmodel="BM25") % 10
    tf_query = br_tf(query_df)
    tf_idf = br_tf_idf(query_df)
    br_BM25 = br_BM25(query_df)
    return tf_query.merge(tf_idf, on=["qid", "docno"]).merge(br_BM25, on=["qid", "docno"]).sort_values(by='score', ascending=False)


my_ordonancer(index, "chemical")


/home/charles/.pyenv/versions/3.10.10/lib/python3.10/site-packages/pyterrier/ops.py:190: FutureWarning: .transform() should be passed a dataframe. Use .search() to execute a single query.
  res = self.transformer.transform(topics_and_res)
/home/charles/.pyenv/versions/3.10.10/lib/python3.10/site-packages/pyterrier/ops.py:190: FutureWarning: .transform() should be passed a dataframe. Use .search() to execute a single query.
  res = self.transformer.transform(topics_and_res)
/home/charles/.pyenv/versions/3.10.10/lib/python3.10/site-packages/pyterrier/ops.py:190: FutureWarning: .transform() should be passed a dataframe. Use .search() to execute a single query.
  res = self.transformer.transform(topics_and_res)


,qid,docid_x,docno,rank_x,score_x,query_x,docid_y,rank_y,score_y,query_y,docid,rank,score,query
0,1,37771,jn5qi1jb,0,9.0,chemical,37771,0,6.809923,chemical,37771,0,12.426309,chemical
1,1,134305,0smev8vt,1,8.0,chemical,134305,2,6.736806,chemical,134305,2,12.292890,chemical
3,1,142104,77c9ohxj,4,7.0,chemical,142104,3,6.700190,chemical,142104,3,12.226076,chemical
4,1,56631,sps45fj5,9,6.0,chemical,56631,6,6.380524,chemical,56631,6,11.642770,chemical
2,1,2524,ifebw24e,2,7.0,chemical,2524,9,6.269341,chemical,2524,9,11.439890,chemical


## Reformulation de requêtes 

Il est également possible de mettre en place des pipelines de [reformulation de requêtes](https://pyterrier.readthedocs.io/en/latest/rewrite.html).

In [12]:
bo1 = pt.rewrite.Bo1QueryExpansion(index)
dph = pt.BatchRetrieve(index, wmodel="DPH")
pipeline = dph >> bo1 >> dph
pipeline.search("chemical reactions")

,qid,docid,docno,rank,score,query_0,query
0,1,147193,ei4rb8fr,0,20.728032,chemical reactions,applypipeline:off chemic^1.457741865 reaction^1.302919515 hazard^0.306805986 reactiv^0.283917121 explos^0.245621780 exotherm^0.236822495 bioproces...
1,1,171636,v3blnh02,1,20.684654,chemical reactions,applypipeline:off chemic^1.457741865 reaction^1.302919515 hazard^0.306805986 reactiv^0.283917121 explos^0.245621780 exotherm^0.236822495 bioproces...
2,1,18717,iavwkdpr,2,11.160010,chemical reactions,applypipeline:off chemic^1.457741865 reaction^1.302919515 hazard^0.306805986 reactiv^0.283917121 explos^0.245621780 exotherm^0.236822495 bioproces...
3,1,20409,1g9kmpdi,3,10.702739,chemical reactions,applypipeline:off chemic^1.457741865 reaction^1.302919515 hazard^0.306805986 reactiv^0.283917121 explos^0.245621780 exotherm^0.236822495 bioproces...
4,1,170863,sj8i9ss2,4,10.347593,chemical reactions,applypipeline:off chemic^1.457741865 reaction^1.302919515 hazard^0.306805986 reactiv^0.283917121 explos^0.245621780 exotherm^0.236822495 bioproces...
...,...,...,...,...,...,...,...
995,1,114233,2cxb65j1,995,3.942073,chemical reactions,applypipeline:off chemic^1.457741865 reaction^1.302919515 hazard^0.306805986 reactiv^0.283917121 explos^0.245621780 exotherm^0.236822495 bioproces...
996,1,183279,8emqwp2o,996,3.942073,chemical reactions,applypipeline:off chemic^1.457741865 reaction^1.302919515 hazard^0.306805986 reactiv^0.283917121 explos^0.245621780 exotherm^0.236822495 bioproces...
997,1,88197,f7g1zacc,997,3.941175,chemical reactions,applypipeline:off chemic^1.457741865 reaction^1.302919515 hazard^0.306805986 reactiv^0.283917121 explos^0.245621780 exotherm^0.236822495 bioproces...
998,1,178340,zsp5v8hv,998,3.938286,chemical reactions,applypipeline:off chemic^1.457741865 reaction^1.302919515 hazard^0.306805986 reactiv^0.283917121 explos^0.245621780 exotherm^0.236822495 bioproces...


L'autre solution est de l'intégrer directement dans la fonction d'ordonnancement. Mais la requête reformulée n'est pas visible et la solution précédente fait prendre conscience de la pipeline faite par le système de RI (ranking >> reformulation >> ranking quand on utilise des modèles basés sur la relevance feedback. ou reformulation >> ranking sinon). Plus d'exemples [ici](https://pyterrier.readthedocs.io/en/latest/rewrite.html).

In [13]:
# modèle DPH avant reformulation de requête
pipelineQE = pt.BatchRetrieve(index, wmodel="DPH", controls={"qemodel" : "Bo1", "qe" : "off"})
pipelineQE.search("chemical reactions")

,qid,docid,docno,rank,score,query
0,1,171636,v3blnh02,0,11.898228,chemical reactions
1,1,147193,ei4rb8fr,1,11.889920,chemical reactions
2,1,18717,iavwkdpr,2,11.688030,chemical reactions
3,1,170863,sj8i9ss2,3,10.143884,chemical reactions
4,1,121217,msdycum2,4,9.860973,chemical reactions
...,...,...,...,...,...,...
995,1,154582,q4neat3x,995,4.070840,chemical reactions
996,1,171895,yjbcvf3o,996,4.070840,chemical reactions
997,1,14781,0it25j3k,997,4.067507,chemical reactions
998,1,70003,a1nc5k51,998,4.067507,chemical reactions


In [14]:
# modèle DPH après reformulation de requête
pipelineQE = pt.BatchRetrieve(index, wmodel="DPH", controls={"qemodel" : "Bo1", "qe" : "on"})
pipelineQE.search("chemical reactions")

,qid,docid,docno,rank,score,query
0,1,147193,ei4rb8fr,0,20.728032,chemical reactions
1,1,171636,v3blnh02,1,20.684654,chemical reactions
2,1,18717,iavwkdpr,2,11.160010,chemical reactions
3,1,20409,1g9kmpdi,3,10.702739,chemical reactions
4,1,170863,sj8i9ss2,4,10.347593,chemical reactions
...,...,...,...,...,...,...
995,1,114233,2cxb65j1,995,3.942073,chemical reactions
996,1,183279,8emqwp2o,996,3.942073,chemical reactions
997,1,88197,f7g1zacc,997,3.941175,chemical reactions
998,1,178340,zsp5v8hv,998,3.938286,chemical reactions


### Evaluation d'un système de recherche d'information

Pour évaluer un système de RI, il est nécessaire d'avoir un jeu de données constitué de requêtes et de jugements de pertinence. 

In [15]:
# exemple de 5 requêtes pour cord19
cord19.get_topics(variant='title').head(5)

[INFO] [starting] https://ir.nist.gov/covidSubmit/data/topics-rnd5.xml
[INFO] [finished] https://ir.nist.gov/covidSubmit/data/topics-rnd5.xml: [00:00] [18.7kB] [14.8MB/s]


,qid,query
0,1,coronavirus origin
1,2,coronavirus response to weather changes
2,3,coronavirus immunity
3,4,how do people die from the coronavirus
4,5,animal models of covid 19


In [16]:
# exemple de jugements de pertinence pour les 5 premières requêtes
cord19.get_qrels().head(5)

[INFO] [starting] https://ir.nist.gov/covidSubmit/data/qrels-covid_d5_j0.5-5.txt
[INFO] [finished] https://ir.nist.gov/covidSubmit/data/qrels-covid_d5_j0.5-5.txt: [00:01] [1.14MB] [904kB/s]


,qid,docno,label,iteration
0,1,005b2j4b,2,4.5
1,1,00fmeepz,1,4
2,1,010vptx3,2,0.5
3,1,0194oljo,1,2.5
4,1,021q9884,1,4


**Exercice 4**

A partir des requêtes et des jugements de pertinence du jeu de données CORD19, Ecrire le code qui permet d'afficher les résultats de la première requête de Cord19. L'affichage fusionnera les colonnes retournées par le BatchRetriever et les colonnes des qrels (merge sur qid et docno pour rajouter label et iteration au tableau). 

In [104]:
def print_pertinence(idx, br):
    idx = str(idx)
    topics = cord19.get_topics(variant="title")
    topic = topics[topics['qid'] == idx]
    jugements = cord19.get_qrels()
    jugement = jugements.loc[jugements['qid'] == idx, 'label']
    res = br(topic).merge(jugements, on=['qid', 'docno'])
    return res
print_pertinence(1, br)

,qid,docid,docno,rank,score,query,label,iteration
0,1,45549,8l411r1w,2,15.0,coronavirus origin,0,1
1,1,13946,2qto9vsb,6,13.0,coronavirus origin,0,3
2,1,41944,5w194etz,11,11.0,coronavirus origin,0,5
3,1,104314,2zaxn6tq,14,11.0,coronavirus origin,0,5
4,1,114693,6ez0u7iq,16,11.0,coronavirus origin,0,5
...,...,...,...,...,...,...,...,...
192,1,33887,b9vkja80,935,4.0,coronavirus origin,0,4.5
193,1,34713,ffmxvb23,940,4.0,coronavirus origin,1,4.5
194,1,43491,9gi2yoqn,954,4.0,coronavirus origin,1,4
195,1,46265,4sfgha4z,963,4.0,coronavirus origin,1,4


Il existe cependant une fonction qui permet de calculer l'efficacité de ces ordonnancements au travers des métriques d'évaluation (map, précision, rappel, ndcg, ...)

In [17]:
pt.Experiment(
    [tfidf],
    cord19.get_topics(variant='title'),
    cord19.get_qrels(),
    eval_metrics=["map", "ndcg"])

,name,map,ndcg
0,BR(TF_IDF),0.180002,0.370767


**Exercice 5**

Réaliser une expérience comparant l'expansion de requêtes avec le modèle Bo1 et basé sur la KL-divergence; L'expérience est réalisée sur TREC CORD19 avec le modèle de référence BM25. Vous devrez construire des pipelines appropriées (plus de détails sur [l'expansion](https://pyterrier.readthedocs.io/en/latest/rewrite.html) et les [expérimentations](https://pyterrier.readthedocs.io/en/latest/experiments.html)).

Quelles approches entraînent des augmentations significatives de NDCG et MAP ou autres métriques ?

In [120]:

dph = pt.BatchRetrieve(index, wmodel="DPH")
bo1 = pt.rewrite.Bo1QueryExpansion(index) >> dph
kl = pt.rewrite.KLQueryExpansion(index) >> dph
pipeline = dph >> bo1 >> kl

pt.Experiment(
    [br_bm25],
    pipeline(cord19.get_topics(variant='title')),
    cord19.get_qrels(),
    eval_metrics=["map", "ndcg"])

,name,map,ndcg
0,BR(BM25),0.188889,0.392789
